In [1]:
%load_ext autoreload
import os
import sys

os.environ['TRANSFORMERS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache-new'
os.environ['HF_DATASETS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache-new'
os.environ["OPENAI_API_KEY"]= 'xxx'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sys.path.append('../src-py')

In [2]:
import json
#keys = json.load(open('/mnt/swordfish-pool2/milad/code/llms-as-science-communicators/keys.json'))
keys = json.load(open('../keys.json'))
for key, val in keys.items():
    os.environ[key] = val

In [3]:
import datasets
import json
import os
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import re
from collections import Counter

pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


In [4]:
from huggingface_hub import login

login(os.environ['hf_token'])

In [5]:
output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/'

In [9]:
dpo_ds = datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/new-deepseek-final-conv-ds-with-preference-data-20k/')

In [10]:
dpo_ds

Dataset({
    features: ['paper_id', 'paper_title', 'paper_text', 'prompt', 'completion', 'chosen', 'rejected', 'answer_quality'],
    num_rows: 0
})

In [8]:
dpo_ds

Dataset({
    features: ['paper_id', 'paper_title', 'paper_text', 'prompt', 'completion', 'chosen', 'rejected', 'answer_quality'],
    num_rows: 16431
})

In [18]:
k = 0
print('\n\n'.join(x['content'] for x in dpo_ds['prompt'][k][2:]))
print('------')
print(dpo_ds['prompt'][k][-1]['content'])
print("Answer:", dpo_ds['answer_quality'][k])
print("Chosen:", dpo_ds['chosen'][k])
print("Rejected:", dpo_ds['rejected'][k])

Journalist:   
Dr. [Researcher's Name], thank you for joining us. Your paper introduces a new method called the stairway plot to infer historical human population size fluctuations. Can you explain what the stairway plot method is and how it differs from previous methods?

Researcher:   
Certainly. The stairway plot is a model-flexible method that uses SNP frequency spectra to estimate changes in population size over time. Unlike previous methods that rely on predefined demographic models, the stairway plot allows for a more flexible exploration of population history. It calculates the expected composite likelihood of the SNP frequency spectrum, treating each SNP as an independent locus, which makes it computationally efficient and suitable for large datasets like whole-genome sequences of hundreds of individuals.
------
Researcher:   
Certainly. The stairway plot is a model-flexible method that uses SNP frequency spectra to estimate changes in population size over time. Unlike previou

In [20]:
dpo_ds.push_to_hub('miladalsh/new-deepseek-final-conv-ds-with-preference-data')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


CommitInfo(commit_url='https://huggingface.co/datasets/miladalsh/new-deepseek-final-conv-ds-with-preference-data/commit/2e27cd52feee134dbbbec3c28e29abd827468b44', commit_message='Upload dataset', commit_description='', oid='2e27cd52feee134dbbbec3c28e29abd827468b44', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/miladalsh/new-deepseek-final-conv-ds-with-preference-data', endpoint='https://huggingface.co', repo_type='dataset', repo_id='miladalsh/new-deepseek-final-conv-ds-with-preference-data'), pr_revision=None, pr_num=None)